### 配置文件验证

In [ ]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [1]:
from modelModule.model2 import VAE2
import torch

In [2]:
model = VAE2()
a = torch.randn((10,57))
b = torch.randn((10,57))
model(a,b)

(tensor([[0.5167, 0.4970, 0.5711, 0.4300, 0.4848, 0.5766, 0.5974, 0.5834, 0.5032,
          0.5194, 0.5700, 0.4613, 0.4645, 0.4794, 0.5157, 0.5078, 0.5613, 0.4132,
          0.5194, 0.6070, 0.4825, 0.5692, 0.5214, 0.5841, 0.5791, 0.4488, 0.5451,
          0.4537, 0.6027, 0.3594, 0.5201, 0.5420, 0.4660, 0.4747, 0.5632, 0.5095,
          0.4906, 0.4854, 0.4226, 0.5101, 0.4892, 0.5351, 0.4664, 0.5232, 0.4182,
          0.5010, 0.4596, 0.5711, 0.4648, 0.4447, 0.5046, 0.4859, 0.5115, 0.4472,
          0.4568, 0.5430, 0.4279],
         [0.5181, 0.4814, 0.5512, 0.4271, 0.5035, 0.5897, 0.6157, 0.6029, 0.5023,
          0.5349, 0.5853, 0.4707, 0.4992, 0.4881, 0.5447, 0.5148, 0.4854, 0.4099,
          0.5084, 0.5547, 0.5066, 0.5608, 0.4741, 0.6334, 0.5908, 0.4532, 0.5312,
          0.4460, 0.5797, 0.3763, 0.4916, 0.5652, 0.4747, 0.5277, 0.5366, 0.5163,
          0.4889, 0.4615, 0.4596, 0.5390, 0.5110, 0.5542, 0.5108, 0.5269, 0.4014,
          0.4875, 0.4694, 0.6194, 0.4671, 0.4309, 0.4735, 0.456

### dataloader验证

In [1]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [2]:
dataset = FlatDataset('Spam_test.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [5]:
batch.keys()

dict_keys(['src_data', 'normal_data', 'miss_data', 'miss_matrix'])

### 模型精度测定

In [10]:
from dataModule.dataset1 import FlatDataset
from modelModule.model1 import VAE
from torch.utils.data import DataLoader
from utils import restore_data, result_show, get_missing
import torch

In [12]:
## 模型参数载入
model = VAE(dim=16, nhead=4)
checkpoint_path = '模型参数保存/Letter_lr_0_0001_lr_scheduler_CosineAnnealingLR/version_2/checkpoints/last.ckpt'    
checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
print('载入成功')

载入成功


In [18]:
dataset = FlatDataset('Letter_train.csv', is_test=True, data_norm='minmax_norm') # minmax_norm
index = 102
# src_data, M_matrix, miss_data = dataset[index]['src_data'].reshape(1,-1), dataset[index]['global_miss_matrix'].reshape(1,-1), dataset[index]['global_miss_data'].reshape(1,-1)
src_data = dataset[index]['src_data'].reshape(1,-1)
normal_data = dataset[index]['global_normal_data'].reshape(1,-1)
miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
res_imputed_data = restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val)
result_show(src_data, res_imputed_data, M_matrix)

原始数据 [ 6.  2.  6. 10.  9.]
插补数据 [5.3713654  3.5199894  4.69653413 7.89741933 8.36493373]


array([0.10477243, 0.7599947 , 0.21724431, 0.21025807, 0.07056292])